# Document's Features

#### · Display raw data from cluster

#### · Display raw data and located peaks recorded in cluster 

#### · Find peaks of recorder raw data with the most updated version of the algorithm 

In [1]:
from functions import *

/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#!pip install pymongo==3.13.0

In [3]:
#import util

In [4]:
my_name = 'braulio.ramirez'

In [5]:
logger = logging.getLogger('jhub')
hostname = 'mongos.mongo.svc.cluster.local:27017'
pemkeyfile = '/etc/mongo/jhub-keypem.pem'
sslca = '/etc/mongo/root-ca.pem'

nirscloud_util_meta.init(logger, 'meta', hostname=hostname, ssl=True, cert=pemkeyfile, ca=sslca)

In [6]:
spark_kerberos_principal = my_name + '@BABYNIRS.ORG'

params = {
    'spark_kerberos_principal': spark_kerberos_principal,
}
nirscloud_util_hdfs.init('/etc/jhub/conf/production.ini', params=params)

2023-07-12 15:20:09 [INFO ] cfg#_init_hdfs_kinit@79: after hdfs_kinit: the_stdout: b'' the_stderr: b''
2023-07-12 15:20:09 [INFO ] client#__init__@192: Instantiated <KerberosClient(url='https://hdfs2.babynirs.org:9870;https://hdfs1.babynirs.org:9870;https://hdfs4.babynirs.org:9870')>.


### Pulling the recorded peaks from the cluster

In [7]:
hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929'

In [8]:
kafka_topics = 'nk_rpeak2_NICU'
hdfs_prefix = '/kafka/topics/%s'% (kafka_topics)

In [9]:
def _get_df(hdfs_path, the_id):
    hdfs_path = hdfs_path
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    print(err, df)
    is_valid = df['id'] == the_id
    df = df[is_valid]
    return df

In [10]:
df_II_peaks = _get_df(hdfs_path, 'II')

2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/'.
2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929'.
2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/'.
2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929'.
2023-07-12 15:20:09 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929'.
2023-07-12 15:20:09 [INFO ] client#list@1123: Listing '/kafka/topics/nk_rpeak2_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929'.
2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/'.
2023-07-12 15:20:09 [INFO ] client#status@320: Fetching status for '/kafka/topics/

None        id             start_ns          rec_nano_ts       val      ver  \
0      II  1686376198314000000  1686376199418015371  1.026001  v0.0.23   
1      II  1686376198826000000  1686376199794493203  0.591038  v0.0.23   
2      II  1686376199338000000  1686376200164770511 -0.116190  v0.0.23   
3      II  1686376199338000000  1686376200535670419  0.825738  v0.0.23   
4      II  1686376199850000000  1686376200903335551 -0.066866  v0.0.23   
...    ..                  ...                  ...       ...      ...   
10095  II  1686374381226000000  1686374382120441656 -0.329833  v0.0.23   
10096  II  1686374381226000000  1686374382459361874  1.132980  v0.0.23   
10097  II  1686374381738000000  1686374382798450036  0.174888  v0.0.23   
10098  II  1686374382250000000  1686374383135835153  0.434357  v0.0.23   
10099  II  1686374382250000000  1686374383474825361  0.436551  v0.0.23   

      _device_id  _bed_id   _the_date _hr _patient_id  
0        Procyon  HA11-01  2023-06-10  01     5984

In [11]:
df_II_peaks.sort_values(by=['rec_nano_ts'], inplace=True)
df_II_peaks.reset_index(drop=True, inplace=True)
df_II_peaks['shift_rec_nano_ts'] = df_II_peaks['rec_nano_ts'].shift(-1)
df_II_peaks['dif_rec_nano_ts'] = df_II_peaks['shift_rec_nano_ts'] - df_II_peaks['rec_nano_ts']
df_II_peaks;

In [12]:
values_p = np.asarray(df_II_peaks['val'].to_numpy())
raw_time_p = np.asarray(df_II_peaks['rec_nano_ts'].to_numpy(), dtype = 'int')
time_p = [np.datetime64(int(t),'ns') for t in raw_time_p]
time_pks = np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_p]);

In [13]:
kafka_topics_cs = 'nk_waves_NICU'
hdfs_prefix_cs = '/nirscloud/agg_by_hr3/%s'% (kafka_topics_cs)

In [14]:
def _get_df_cs(hdfs_path, the_id):
    hdfs_path = hdfs_path + '/id=%s' % (the_id)
    full_path = nirscloud_util_hdfs.full_path(hdfs_prefix_cs, hdfs_path)
    err, df = nirscloud_util_hdfs.from_hdfs_path(full_path)
    return df

In [15]:
# Time series need to be concatenated accoording to the amount of time that is going to be analyzed 

# Getting a sample from raw data in cluster

##### When getting a sample the first thing to be considered is the amount of time, and this is determined by the stats that are to be done afterwards.
##### We cacn think in terms of how many windows do we want to analyze or how much time, the length of the window and their overlap.

In [16]:
# The total time of the interval will be defined by the amount of windows one chooses, the overlap and the length of the windows.
time_wind_length = 5; overlap_window = 0.5; 
print('Total time length can be in intervals of ' +str(round(time_wind_length*overlap_window,2))+ ' minutes starting from '+str(time_wind_length))
print('')
print('You can either choose the amount of windows or the total amount of time')

Total time length can be in intervals of 2.5 minutes starting from 5

You can either choose the amount of windows or the total amount of time


In [17]:
#In this block you can enter the total amount of time to analyze or the amount of windows

by_time = True; total_time = 20; windows = 12;


if by_time: 
    by_windows = False
else: by_windows = True

if by_time: 
    if total_time%(time_wind_length*overlap_window) != 0: 
        print('The amount of time entered is not divisible by '+str(time_wind_length*overlap_window))
    windows = int(total_time/(time_wind_length*overlap_window)) - 1 
    print('The amount of time is '+str(int(total_time))+' minutes and the amount of windows is '+str(windows))

    
if by_windows: 
    windows = 12
    total_time = (1-overlap_window)*time_wind_length*(windows) + 15
    print('The amount of time is '+str(int(total_time))+' minutes and the amount of windows is '+str(windows))


The amount of time is 20 minutes and the amount of windows is 7


In [18]:
#This line is important because it establishes what will be the first hour 
first_hour = 1

In [19]:
# Extracting and concatenating dataframes according to time. 

if total_time%60 == 0: 
    amount_hours = int(total_time/60)
else:
    amount_hours = int((total_time - total_time%60)/60) + 1 

if amount_hours == 1:
    print('Extracting '+str(int(amount_hours))+' hour')
else: 
    print('Extracting '+str(int(amount_hours))+' hours')

df_dict = {}; time_cts =[]; values_cs = [];
hours = np.arange(first_hour,first_hour + amount_hours)
lead = 'MDC_ECG_ELEC_POTL_II'

for h in hours:
    if len(str(h)) == 1:
        hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=0' + str(h)+'/_patient_id=5984929'
    else: 
        hdfs_path = '_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=' + str(h)+'/_patient_id=5984929'

    df_cs = _get_df_cs(hdfs_path, lead)
    print('Extracted hour: '+str(h))
    df_dict['hour '+str(h)] = df_cs
    
final_df_cs = pd.concat(df_dict)


final_df_cs.sort_values(by=['_milli_ts'], inplace=True)
final_df_cs.reset_index(drop=True, inplace=True)
final_df_cs['shift_milli_ts'] = final_df_cs['_milli_ts'].shift(-1)
final_df_cs['diff_milli_ts'] = final_df_cs['shift_milli_ts'] - final_df_cs['_milli_ts']

values_cs = np.asarray(final_df_cs['val'].to_numpy())
raw_time_cs = np.asarray(final_df_cs['_milli_ts'].to_numpy(), dtype = 'int')
time_cs = [np.datetime64(int(t),'ms') for t in raw_time_cs]
time_cts = np.concatenate([time_cts, np.array([(pd.Timestamp(t).hour*60*60*1000) + (pd.Timestamp(t).minute*60*1000) + 
                 (pd.Timestamp(t).second*1000) + (pd.Timestamp(t).microsecond/1000) for t in time_cs])]);


missing_info = final_df_cs[final_df_cs['diff_milli_ts']>4]
missing_ids = missing_info.index.to_list()

if len(missing_ids)>0:
    print('')
    print('There is missing data in the following indices')
for missing_idx in missing_ids: 
    print('')
    print('Time '+str(round((missing_idx*4/1000)/60,2))+', Index '+str(missing_idx)+' is missing '+str(final_df_cs['diff_milli_ts'][missing_idx])+' milliseconds ahead')
    zeros = np.zeros(int(final_df_cs['diff_milli_ts'][missing_idx]/4))
    missing_time = np.arange(time_cts[missing_idx]+4,time_cts[missing_idx+1],4)
    time_cts = np.insert(time_cts,missing_idx+1,missing_time)
    values_cs = np.insert(values_cs,missing_idx,zeros)
    
time_cts = time_cts - time_cts[0]

print('Finished')

2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/'.
2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/'.


Extracting 1 hour


2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-12 15:20:13 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-12 15:20:13 [INFO ] client#list@1123: Listing '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patient_id=5984929/id=MDC_ECG_ELEC_POTL_II'.
2023-07-12 15:20:13 [INFO ] client#status@320: Fetching status for '/nirscloud/agg_by_hr3/nk_waves_NICU/_device_id=Procyon/_bed_id=HA11-01/_the_date=2023-06-10/_hr=01/_patien

Extracted hour: 1
Finished


# Classes

In [20]:
time_pks = time_pks - time_cts[0]

In [21]:
class raw_sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4) + int(self.min_sec*1000/4)
        self.max_time = int((self.max_min*60*1000)/4) + int(self.max_sec*1000/4) + 1
        self.pks_time_min = (self.min_min*60*1000) + (self.min_sec*1000)
        self.pks_time_max = (self.max_min*60*1000) + (self.max_sec*1000)
        self.pks_time = time_pks[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        self.pks_values = values_p[(self.pks_time_min > time_pks).argmin():(time_pks < self.pks_time_max).argmin()]
        
        self.time = time_cts[self.min_time:self.max_time]
        self.values = df_II_cs['val'].to_numpy()[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))
        self.fig.add_trace(go.Scatter(mode='markers', marker= dict(color='red'),x=self.pks_time, y=self.pks_values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)
        

In [22]:
class sample_():
    
    def __init__(self,min_min,min_sec,max_min,max_sec,seconds_per_label,find_the_peaks,inverted):
        
        self.min_min = min_min
        self.min_sec = min_sec
        self.max_min = max_min
        self.max_sec = max_sec
        self.seconds_per_label = seconds_per_label

        self.min_time = int((self.min_min*60*1000)/4 + int(self.min_sec*1000)/4) 
        self.max_time = int((self.max_min*60*1000)/4 + int(self.max_sec*1000)/4) + 1

        
        time_zero = raw_time_cs[0]
        self.time = raw_time_cs[self.min_time:self.max_time] - time_zero
        if inverted:
            self.values = - values_cs[self.min_time:self.max_time]
        else: 
            self.values = values_cs[self.min_time:self.max_time]

        self.fig = go.Figure()
        self.fig.add_trace(
                      go.Scatter(line= dict(color='blue'),x=self.time, y=self.values,showlegend=False))

        self.ticks_values = []
        x = self.time[0]
        while x < self.time[-1]:
            self.ticks_values = np.append(self.ticks_values,x)
            x = 1000*self.seconds_per_label + x

        self.time_format_axis = time_f(self.ticks_values)
        
        if find_the_peaks:
            self.peaks, _ = find_peaks(self.values, distance=100)
            self.fig.add_trace(
                      go.Scatter(mode='markers',x=self.time[self.peaks], y=self.values[self.peaks],showlegend=False))


In [30]:
sample1 = sample_(0,0,20,0,10,False,False)
fig = sample1.fig
fig.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1250,
                  height=600,showlegend=False,title="Raw Data ", xaxis_title="Time", yaxis_title= 'Signal', 
                  font=dict(family="Avenir",size=14,color="Black"));

In [32]:
#fig.show()

# Pre processing, baseline wander removal through moving average subtraction

$$y[n]_{Moving Average} = \frac{1}{M_2 + 1}(x[n]-x[n-M_2-1]) + y[n-1]$$


In [33]:
# This block proves that the difference equation has the same result with the convolution, data has been preprocessed 
# and pan tompkins algorithm can take place now. 

upper_limit = 10000
w = 15
mean = average_filter(w,sample1.values)
yn, cut_pts = moving_average(w,sample1.values)

In [34]:
# Comparing different window lengths for pre processing stage

fig_pre_p = go.Figure()

baseline_removal = {}
prepro_data = {}
cut_pts ={}
x = sample1.time
for i in range(10,11):    
    print(i)
    baseline_removal['Window '+str(i)], cut_pts['Window '+str(i)] = moving_average(i,sample1.values)
    prepro_data['Window '+str(i)] =  sample1.values[0:len(x)-cut_pts['Window '+str(i)]] - baseline_removal['Window '+str(i)]
    fig_pre_p.add_trace(go.Scatter(line= dict(color='#e65400'), x=x[0:len(x)-cut_pts['Window '+str(i)]], y = prepro_data['Window '+str(i)],
                                   name = 'Baseline Removal Window '+str(i), showlegend=False, visible = False))
    #fig_pre_p.add_trace(go.Scatter(line= dict(color='blue'),    x=sample1.time, y=sample1.values,name='Raw Data',showlegend=False, visible = False))

    
fig_pre_p.data[0].visible = True


steps = []
start = 0
for t in range(10,11):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_pre_p.data)},
              {"title":  'Baseline Wander Removal      Moving Average Window: '+str(t)}],  # layout attribute
    )
    
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    #step["args"][0]["visible"][start+1] = True

    
    start = start+1
    steps.append(step)

    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig_pre_p.update_layout(sliders=sliders);
fig_pre_p.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600);

10


In [35]:
#fig_pre_p.show()

In [36]:
#Comparing the Raw data and Pre processed data in subplots

i =  10
fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Raw Data"," Mean Subtraction Filter "))

fig3.add_trace(go.Scatter(line = dict(color='blue'), x=sample1.time, y=sample1.values,name='Raw Data',showlegend=False),row=1, col=1)
fig3.add_trace(go.Scatter(x=x[0:len(x)-cut_pts['Window '+str(i)]], y= prepro_data['Window '+str(i)],
                                   name = 'Baseline Removal Window '+str(i), showlegend=False,),row=2, col=1)



fig3.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000],yaxis_range=[-0.6,0.65])
fig3.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600);
fig3.update_layout(xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600);
fig3.update_layout(height=700,width = 1300);

#### A window of 10 is chosen for the mean subtraction, based oon high noise intervals, need too look deeper into this

## Pan Tompkins starts here

In [37]:
#time_dif = np.unique(np.diff(sample1.time))/np.timedelta64(1, "ms")
time_dif = 4/1000
sampling_freq = 1/time_dif
print('Sampling Frequency: '+str(sampling_freq)+'Hz')

Sampling Frequency: 250.0Hz


In [93]:
lowcut = 3; highcut = 100;
nyquist_freq = 0.5 * sampling_freq
"""
low = lowcut / nyquist_freq
high = highcut / nyquist_freq
b, a = butter(2, [low, high], btype="band")
"""

'\nlow = lowcut / nyquist_freq\nhigh = highcut / nyquist_freq\nb, a = butter(2, [low, high], btype="band")\n'

## Bandpass Filter 

In [39]:
"""
fr_r = go.Figure()
fs = 250
trans_width = 2.88
band = [3, 18]  # Desired pass band, Hz
filter_size= np.arange(100,200,1)
edges = [0, band[0] - trans_width, band[0], band[1],
         band[1] + trans_width, 0.5*fs]
trans_width = 2.8    # Width of transition from pass to stop, Hz
taps = {}

for i in range(0,len(filter_size)):

    numtaps=filter_size[i]      # Size of the FIR filter.

    taps['size: '+str(filter_size[i])] = signal.remez(numtaps, edges, [0, 1, 0], fs=fs)
    w, h = signal.freqz(taps['size: '+str(filter_size[i])], [1], worN=2000, fs=fs)
    fr_r.add_trace(go.Scatter(x=w,y=20*np.log10(np.abs(h))))
    
fr_r.data[5].visible = True

steps = []
for tw in range(0,len(filter_size)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fr_r.data)},
              {"title": 'Size of Filter: '+ str(filter_size[tw])}],  # layout attribute
    )
    step["args"][0]["visible"][tw] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fr_r.update_layout(sliders=sliders);


fr_r.update_layout(
    xaxis = dict(tickmode = 'array',tickvals = [np.log10(1),np.log10(2),np.log10(3),np.log10(4),np.log10(5),np.log10(6),np.log10(7),np.log10(8),np.log10(9),
                    np.log10(10),np.log10(15),np.log10(20)],
        ticktext = ['1', '2', '3', '4', '5', '6','7','8','9','10','15','20']))

fr_r.update_layout(autosize=False,width=1200, height=600, xaxis_title="w", yaxis_title= '20log( |H(s)| ) (dB)' ,
                      font=dict(family="Avenir",size=16,color="Black"))

fr_r.update_layout(xaxis_range=[0,150], yaxis_range=[-50,10]);

"""

'\nfr_r = go.Figure()\nfs = 250\ntrans_width = 2.88\nband = [3, 18]  # Desired pass band, Hz\nfilter_size= np.arange(100,200,1)\nedges = [0, band[0] - trans_width, band[0], band[1],\n         band[1] + trans_width, 0.5*fs]\ntrans_width = 2.8    # Width of transition from pass to stop, Hz\ntaps = {}\n\nfor i in range(0,len(filter_size)):\n\n    numtaps=filter_size[i]      # Size of the FIR filter.\n\n    taps[\'size: \'+str(filter_size[i])] = signal.remez(numtaps, edges, [0, 1, 0], fs=fs)\n    w, h = signal.freqz(taps[\'size: \'+str(filter_size[i])], [1], worN=2000, fs=fs)\n    fr_r.add_trace(go.Scatter(x=w,y=20*np.log10(np.abs(h))))\n    \nfr_r.data[5].visible = True\n\nsteps = []\nfor tw in range(0,len(filter_size)):\n    step = dict(\n        method="update",\n        args=[{"visible": [False] * len(fr_r.data)},\n              {"title": \'Size of Filter: \'+ str(filter_size[tw])}],  # layout attribute\n    )\n    step["args"][0]["visible"][tw] = True  # Toggle i\'th trace to "visible

In [40]:
fig_bp = go.Figure()
i = 10
bandstop = np.arange(3,4)
fs = 250
trans_width = 2.88
taps = {}
numtaps = 152

for bs in bandstop:
    band = [bs, 100]
    edges = [0, band[0] - trans_width, band[0], band[1],
             band[1] + trans_width, 0.5*fs]
    
    taps['Bandstart: '+str(bs)] = signal.remez(numtaps, edges, [0, 1, 0], fs=fs)
    
    
    x = np.append(prepro_data['Window '+str(i)],np.zeros(len(taps['Bandstart: '+str(bs)])))
    y_bp = np.zeros(len(prepro_data['Window '+str(i)])+len(taps['Bandstart: '+str(bs)]))

    for j in range(0,len(prepro_data['Window '+str(i)])+len(taps['Bandstart: '+str(bs)])): 
        sum = 0
        for k in range(0,len(taps['Bandstart: '+str(bs)])):
            sum = (x[j-k]*taps['Bandstart: '+str(bs)][k]) + sum
        y_bp[j] = sum

    y_bp=np.delete(y_bp,[np.arange(0,int(len(taps['Bandstart: '+str(bs)])/2))])
    fig_bp.add_trace(go.Scatter(x=sample1.time, y=y_bp,visible=False))

fig_bp.data[0].visible = True

steps = []
for bps in range(0,len(bandstop)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig_bp.data)},
              {"title": ' Mean Removal Windoow: '+str(i)+'      Bandstart: '+ str(bandstop[bps])}],  # layout attribute
    )
    step["args"][0]["visible"][bps] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]

fig_bp.update_layout(sliders=sliders);
fig_bp.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis)); 
fig_bp.update_layout(autosize=False,width=1200, height=600, xaxis_title="Time", yaxis_title= 'Signal' ,
                      font=dict(family="Avenir",size=16,color="Black"));

In [41]:
#fig_bp.show()

#### Chosen FIR Filter

In [42]:
i = 10
x = np.append(prepro_data['Window '+str(i)],np.zeros(len(taps['Bandstart: 3'])))
y_bp = np.zeros(len(prepro_data['Window '+str(i)])+len(taps['Bandstart: 3']))

for j in range(0,len(prepro_data['Window '+str(i)])+len(taps['Bandstart: 3'])): 
    sum = 0
    for k in range(0,len(taps['Bandstart: 3'])):
        sum = (x[j-k]*taps['Bandstart: 3'][k]) + sum
    y_bp[j] = sum
    
y_bp=np.delete(y_bp,[np.arange(0,int(len(taps['Bandstart: 3'])/2))])


In [43]:
#Comparing the Mean Subtraction and Bandpass in subplots

i = 10
fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=(" Mean Subtraction Filter "," Bandpass "))

fig3.add_trace(go.Scatter(line=dict(color = 'red'),x=sample1.time, y = y_bp,name='Raw Data',showlegend=False),row=2, col=1)
fig3.add_trace(go.Scatter(x=x[0:len(x)-cut_pts['Window '+str(i)]], y= prepro_data['Window '+str(i)],
                                   name = 'Baseline Removal Window '+str(i), showlegend=False,),row=1, col=1)



fig3.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000])
fig3.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600);
fig3.update_layout(xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600)
fig3.update_layout(height=700,width = 1300);

In [52]:
#fig3.show()

In [45]:
#Transfer Function for IIR
"""
x0 =np.append(prepro_data['Window '+str(6)],np.zeros(len(b)))
dif_y = np.zeros(len(x0))

for i in range(0,len(prepro_data['Window 15'])):
    dif_y[i] = (1/a[0]) * (b[0]*x[i] + b[1]*x[i-1] + b[2]*x[i-2] + b[3]*x[i-3] + b[4]*x[i-4] - 
                       (a[1]*dif_y[i-1] + a[2]*dif_y[i-2] + a[3]*dif_y[i-3] + a[4]*dif_y[i-4]))
    

dif_y = np.delete(dif_y,np.arange(len(dif_y)-len(b),len(dif_y),1))
"""

"\nx0 =np.append(prepro_data['Window '+str(6)],np.zeros(len(b)))\ndif_y = np.zeros(len(x0))\n\nfor i in range(0,len(prepro_data['Window 15'])):\n    dif_y[i] = (1/a[0]) * (b[0]*x[i] + b[1]*x[i-1] + b[2]*x[i-2] + b[3]*x[i-3] + b[4]*x[i-4] - \n                       (a[1]*dif_y[i-1] + a[2]*dif_y[i-2] + a[3]*dif_y[i-3] + a[4]*dif_y[i-4]))\n    \n\ndif_y = np.delete(dif_y,np.arange(len(dif_y)-len(b),len(dif_y),1))\n"

### AN FIR FILTER WAS MADE WITH NUMBER OF TAPS 152

#### FROM NOW ON THE DATA HAS BEEN FILTERED WITH AN FIR BANDPASS AND CAN CONTINUE WITH THE REST OF THE STEPS IN PAN TOMPKINS

In [46]:
i = str(10)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(line= dict(color='blue', width =2),x=sample1.time, y = sample1.values,name='Raw Data',showlegend=True))
fig2.add_trace(go.Scatter(line= dict(width = 2),x=sample1.time, y = prepro_data['Window '+i],name='Preprocessed Data',showlegend=True))
fig2.add_trace(go.Scatter(line= dict(width = 2),x=sample1.time, y = y_bp,name='FIR Filter',showlegend=True))
fig2.update_layout(xaxis_range=[79000,85000],xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis), 
                    autosize=False,width=1200, height=600, title='Band Pass Filtered and Raw Data', 
                    xaxis_title="Time", yaxis_title= 'Signal');


In [47]:
#fig2.show()

### Difference Equation for derivative

#### y[n] = (1/8T) (-x[n-2] - 2x[n-1] + 2x[n+1] + x[n+2])

In [48]:
#Difference Equation

der1 = np.zeros(len(y_bp))

x = np.append(y_bp,np.zeros(2))

for i in range(0,len(y_bp)):
    der1[i] = (1/8) * (-(x[i-2]) - (2*x[i-1]) + (2*x[i+1]) + (x[i+2]))


In [49]:
# Bandpass and Derivative

fig3 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Bandpass Filter",'Derivative Filter '))

#fig3.add_trace(go.Scatter(line= dict(color='green'),x=sample1.time, y = der1,name='Derivative Filter',showlegend=False),row=1, col=1)

fig3.add_trace(go.Scatter(line= dict(color='red'),x=sample1.time, y = y_bp,name='Bandpass',showlegend=False),row=1, col=1)
fig3.add_trace(go.Scatter(line= dict(color='Green'),x=sample1.time, y = der1,name='Derivative',showlegend=False),row=2, col=1)
#fig3.add_trace(go.Scatter(line= dict(color='Blue'),x=sample1.time, y = der2,name='Derivative',showlegend=False),row=3, col=1)

fig3.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis))

#fig3.update_layout(xaxis_range=[0,1000], xaxis2_range=[0,85000])
fig3.update_layout(height=750,width = 1300);

### Squaring

In [50]:
squared = (der1) ** 2

In [51]:
#Derivative and Squared


fig4 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Derivative Filter     Average Window: 6      Bandstart: 22"," Squared        Average Window: 6         Bandstart: 22 "))

#fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=1, col=1)

fig4.add_trace(go.Scatter(line= dict(color='green'),x=sample1.time, y = der1,name='Derivative',showlegend=False),row=1, col=1)
fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=2, col=1)

fig4.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis))

fig4.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000], yaxis2_range=[-0.001,0.016])
fig4.update_layout(height=700,width = 1300);

## Moving Integration

#### The moving window integration difference equation is given by 
#### $$y[n] = (x[n-(N-1)] + x[n-(N-2)] + .... + x[n])$$

##### The following is the moving integration difference equation implemented through a loop just to test and compare later on with a constructed function.

In [53]:
y_ma = np.zeros(len(squared))
y_mi = np.zeros(len(squared))

# N = window_length
N = 11
squared_y = np.append(squared,np.zeros(N-1))
for n in range(0,len(y_ma)):
    y_ma[n] = (1/N) * (squared_y[n-(N-1)] + squared_y[n-(N-2)] + squared_y[n-(N-3)] + squared_y[n-(N-4)] + squared_y[n-(N-5)] + squared_y[n-(N-6)] + 
                       squared_y[n-(N-7)] + squared_y[n-(N-8)] + squared_y[n-(N-9)] + squared_y[n-(N-10)] + squared_y[n])
    y_mi[n] = (squared_y[n-(N-1)] + squared_y[n-(N-2)] + squared_y[n-(N-3)] + squared_y[n-(N-4)] + squared_y[n-(N-5)] + squared_y[n-(N-6)] + 
                       squared_y[n-(N-7)] + squared_y[n-(N-8)] + squared_y[n-(N-9)] + squared_y[n-(N-10)] + squared_y[n])

In [54]:
# The constructed function
prueba = dif_eq_window_integration(squared,11,False)

In [55]:
# This just doublechecks that the function works

fig5 = go.Figure()

fig5.add_trace(go.Scatter(mode = 'lines', line= dict(color='green'),x=sample1.time, y=squared,name='Squared',showlegend=True))
fig5.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue',width = 4),x=sample1.time, y=y_ma,name='Moving Window Average',showlegend=True))
fig5.add_trace(go.Scatter(mode = 'lines', line= dict(color='red',width = 4),x=sample1.time, y=prueba, name='Prueba',showlegend=True))
fig5.add_trace(go.Scatter(mode = 'lines', x=sample1.time, y=y_mi,name='Moving Window Integration',showlegend=True))


fig5.update_layout(xaxis_range=[79000,85000],xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis), 
                    autosize=False,width=1250, height=600, title="Squared and Moving Window Integration WL: " + str(N), 
                    xaxis_title="Time", yaxis_title= 'Signal', font=dict(family="Avenir",size=14,color="Black"));

## Looking to different window lengths in the window integration difference equation

In [56]:
# Moving Window Integration

fig6 = go.Figure()

y_mw_dict ={}
lengths = np.arange(10,35,1)
peaks_trial = {}

for N in lengths: 
    y_mw_dict['MW Integration W: '+str(N)] = dif_eq_window_integration(squared,N,False)
    #peaks_trial['MWI W peaks: '+str(N)], _ = find_peaks(y_mw_dict['MW Integration W: '+str(N)],distance = 100)
    fig6.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue'),x=sample1.time, y=y_mw_dict['MW Integration W: '+str(N)],
                               name = 'Differece Equation',showlegend=False, visible =False))
    
fig6.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig6.data)},
              {"title": 'Moving Window Integration           Window Length: ' + str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True # Toggle i'th trace to "visible"
    #step["args"][0]["visible"][start + 1] = True # Toggle i'th trace to "visible"
    #start = start +2

    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig6.update_layout(sliders=sliders);
fig6.update_layout(xaxis_range=[79000,85000]);
fig6.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600); 


In [57]:
#fig6.show()

In [58]:
# Moving Window Integration and Squared THIS IS BASICALLY THE SAME AS THE ABOVE BUT WITH THE SQUARED

fig7 = go.Figure()

y_mw_dict ={};


for N in lengths: 
    y_mw_dict['MW Integration W: '+str(N)] = dif_eq_window_integration(squared,N,False)
    fig7.add_trace(go.Scatter(mode = 'lines', line= dict(color='orange'),x=sample1.time, y=squared,
                               name = 'Squared',showlegend=True,visible=False))
    fig7.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue',width=3),x=sample1.time, y=y_mw_dict['MW Integration W: '+str(N)],
                               name = 'Differece Equation',showlegend=True,visible=False))
    
fig7.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig7.data)},
              {"title": 'Moving Window Integration           Window Length: ' + str(lengths[i])}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True # Toggle i'th trace to "visible"
    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig7.update_layout(sliders=sliders);
fig7.update_layout(xaxis_range=[79000,85000],yaxis_range = [0,0.014]);
fig7.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1250,height=650); 


In [59]:
#fig7.show()

In [60]:
# Squared and Moving Window Integration

fig4 = make_subplots(
    rows=2, cols=1,
    subplot_titles=("Squared"," Moving Window Integration "))

#fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=1, col=1)

fig4.add_trace(go.Scatter(line= dict(color='#0a4345'),x=sample1.time, y = y_mw_dict['MW Integration W: 25'],name='MWI',showlegend=False),row=2, col=1)
fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=1, col=1)

fig4.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis))

fig4.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000], yaxis_range=[-0.001,0.0155], yaxis2_range=[-0.001,0.085])
fig4.update_layout(height=800,width = 1300);

In [62]:
# Moving Window Integration and its peaks

fig8 = go.Figure()
fiducial_point_dict = {}

for N in lengths: 
    fig8.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue'),x=sample1.time, y=y_mw_dict['MW Integration W: '+str(N)],
                               name = 'MWI length: '+str(N),showlegend=False,visible=False))
    
    fiducial_point_dict['Window Length: '+str(N)] = find_fiducial_point(y_mw_dict['MW Integration W: '+str(N)],sample1.time,3,0.8,N)

    fig8.add_trace(go.Scatter(mode = 'markers', marker= dict(size = 10, color='red'),x=sample1.time[fps],
                              y=y_mw_dict['MW Integration W: '+str(N)][fps],
                               name = 'Peaks_function',showlegend=False,visible=False))
    
fig8.data[5].visible = True

steps = []
start = 0
for length in lengths:
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig8.data)},
              {"title": 'Moving Window Integration           Window Length: ' + str(length) + '         Peaks Found: '+str(len(fiducial_point_dict['Window Length: '+str(length)]))}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True # Toggle i'th trace to "visible"

    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig8.update_layout(sliders=sliders);
fig8.update_layout(xaxis_range=[79000,85000]);
fig8.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600); 

In [63]:
#fig8.show()

In [64]:
# The algorithm needs to store maybe 5 peaks to learn whats a good height to be consider a peak in the moving window integration. 

In [65]:
# A very simple strategy is to allow for 3 segments of one second or maybe 0.7 seconds to go by and get the maximum value of each segment in 
# that way we have a sample of what a maximum looks like

In [66]:
# Moving Window Integration and Raw Peaks
spline = 4
res = 1000
max_x, max_y, peaks_x, peaks_y, interpolated_peak_x, interpolated_peak_y,max_ids = interpolation_spline(fiducial_point_dict['Window Length: 25'], 
                                                                                               sample1.time, sample1.values, spline, res)
i = 10
fig4 = make_subplots(
    rows=3, cols=1,
    subplot_titles=("Moving Window Integration"," Raw Peaks ", 'Mean Subtraction'))

#fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=1, col=1)

fig4.add_trace(go.Scatter(line= dict(color='#0a4345'),x=sample1.time, y = y_mw_dict['MW Integration W: 25'],name='MWI',showlegend=False),row=1, col=1)
fig4.add_trace(go.Scatter(mode = 'markers',x=sample1.time[fiducial_point_dict['Window Length: 25']], 
                          y = y_mw_dict['MW Integration W: 25'][fiducial_point_dict['Window Length: 25']],
                          name='MWI',showlegend=False),row=1, col=1)
fig4.add_trace(go.Scatter(x=x[0:len(x)-cut_pts['Window '+str(i)]], y= prepro_data['Window '+str(i)],
                                   name = 'Baseline Removal Window '+str(i), showlegend=False,),row=3, col=1)


fig4.add_trace(go.Scatter(line= dict(color='blue'),x=sample1.time, y = sample1.values,name='Peaks in MWI',showlegend=False),row=2, col=1)
fig4.add_trace(go.Scatter(mode='markers',marker=dict(color='red'), x=peaks_x, y = peaks_y, name='Peaks In Raw Data',showlegend=False),row=2, col=1)

fig4.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis3 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis))

fig4.update_layout(xaxis_range=[79000,85000], xaxis2_range=[79000,85000],xaxis3_range=[79000,85000], yaxis_range=[-0.001,0.085], yaxis2_range=[-0.5,0.68])
fig4.update_layout(height=1200,width = 1300);

# Analytical Solution for peaks in interpolation method

In [67]:
"""
f = go.Figure()

f.add_trace(go.Scatter(line = dict(color = 'blue'), x=sample1.time,y=sample1.values, name='Raw Data'))
f.add_trace(go.Scatter(mode='markers',marker=dict(color = '#26E31A', size = 8), x = max_x,y=max_y,name='Analytical Solution'))
f.add_trace(go.Scatter(mode='markers',marker=dict(color = 'green' ,size = 6), x = peaks_x, y=peaks_y, name=' Collected Peak Interpolation '+str(res)+'Hz'))

for h in range(0,len(max_x)):
    f.add_trace(go.Scatter(mode='lines',line=dict(color='#FE4384'), x = interpolated_peak_x['peak: '+str(h)], 
                               y = interpolated_peak_y['peak: '+str(h)], showlegend= False))


f.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,
                     width=1250,height=600)

f.update_layout(title="Pan Tompkins Interpolated Peaks       Moving Average Window = 14 samples     Analytical Solution ", 
                         xaxis_title="Time", yaxis_title= 'Signal', font=dict(family="Avenir",size=16,color="Black"));
                         """

'\nf = go.Figure()\n\nf.add_trace(go.Scatter(line = dict(color = \'blue\'), x=sample1.time,y=sample1.values, name=\'Raw Data\'))\nf.add_trace(go.Scatter(mode=\'markers\',marker=dict(color = \'#26E31A\', size = 8), x = max_x,y=max_y,name=\'Analytical Solution\'))\nf.add_trace(go.Scatter(mode=\'markers\',marker=dict(color = \'green\' ,size = 6), x = peaks_x, y=peaks_y, name=\' Collected Peak Interpolation \'+str(res)+\'Hz\'))\n\nfor h in range(0,len(max_x)):\n    f.add_trace(go.Scatter(mode=\'lines\',line=dict(color=\'#FE4384\'), x = interpolated_peak_x[\'peak: \'+str(h)], \n                               y = interpolated_peak_y[\'peak: \'+str(h)], showlegend= False))\n\n\nf.update_layout(xaxis = dict(tickmode = \'array\', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,\n                     width=1250,height=600)\n\nf.update_layout(title="Pan Tompkins Interpolated Peaks       Moving Average Window = 14 samples     Analytical Solution ", \n          

In [68]:
#f.show()

In [69]:
# Raw Peaks and Third Order Spline Peaks 
max_x, max_y, peaks_x, peaks_y, interpolated_peak_x, interpolated_peak_y, max_idxs = interpolation_spline(fiducial_point_dict['Window Length: 25'], 
                                                                                               sample1.time, sample1.values, spline, res)

#fig4 = make_subplots(
#    rows=2, cols=1,
#    subplot_titles=("Raw Peaks"," Analytical Solution "))

fig4 = go.Figure()

#fig4.add_trace(go.Scatter(line= dict(color='orange'),x=sample1.time, y = squared,name='Squared',showlegend=False),row=1, col=1)
#fig4.add_trace(go.Scatter(line= dict(color='blue'), x=sample1.time, y = sample1.values,name='Peaks in Raw Data',showlegend=False),row=2, col=1)
#fig4.add_trace(go.Scatter(mode='markers',marker=dict(color='red', size=9), x= sample1.time[max_idxs], y = sample1.values[max_idxs], name='Peaks In Raw Data',showlegend=False),row=2, col=1)

fig4.add_trace(go.Scatter(line= dict(color='blue'), x=sample1.time, y = sample1.values,name='Peaks in MWI',showlegend=False))


for j in range(0,len(interpolated_peak_x)):
    fig4.add_trace(go.Scatter(line= dict(color='green'), x=interpolated_peak_x['peak: '+str(j)], y = interpolated_peak_y['peak: '+str(j)], name='Peaks in MWI',showlegend=False))

    
fig4.add_trace(go.Scatter(mode='markers',marker=dict(color='red', size=9), x=max_x, y = max_y, name='Peaks In Raw Data',showlegend=False))
fig4.add_trace(go.Scatter(mode='markers',marker=dict(color='orange', size=7), x=sample1.time[max_idxs], y = sample1.values[max_idxs], name='Raw Peaks',showlegend=False))

fig4.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),
                    xaxis2 = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis));
fig4.update_layout(xaxis_range=[79000,85000], yaxis_range=[-0.5,0.68])
fig4.update_layout(height=600,width = 1300);

In [70]:
#fig4.show()

#### Time Differences with different amount of control points and re sampling frequency 

## Interpolation with different window lenghts in the moving window integration

In [71]:
interp = go.Figure()
spline = 4
max_x_dict= {};max_y_dict= {}
res = 1000 


for N in lengths: 
    
    max_x_dict['Window: '+str(N)], max_y_dict['Window: '+str(N)], peaks_x, peaks_y, interpolated_peak_x, interpolated_peak_y, max_idx = interpolation_spline(
        fiducial_point_dict['Window Length: '+str(N)], sample1.time, sample1.values, spline, res) 
    
    interp.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue'),x=sample1.time, y=sample1.values,showlegend=False,visible=False))
    interp.add_trace(go.Scatter(mode = 'markers', x= max_x_dict['Window: '+str(N)], y=max_y_dict['Window: '+str(N)],
                               name = 'Peaks ',showlegend=False,visible=False))
    

    
interp.data[5].visible = True

steps = []
start = 0
for length in lengths:
    step = dict(
        method="update",
        args=[{"visible": [False] * len(interp.data)},
              {"title": 'Peaks Found           Window Length: ' + str(length)}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True # Toggle i'th trace to "visible"
    start = start + 2
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
interp.update_layout(sliders=sliders);
#interp.update_layout(xaxis_range=[79000,85000]);
interp.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1300,height=600);


/home/shared/braulio/ecg_algorithm/functions.py:232: RuntimeWarning:

invalid value encountered in sqrt

/home/shared/braulio/ecg_algorithm/functions.py:233: RuntimeWarning:

invalid value encountered in sqrt



In [72]:
#interp.show()

## Mean Subtraction and wavelet transform 

### We do the following just to check how good are the PT results 

In [73]:
mean = average_filter(15,sample1.values)

In [74]:
mean_peaks,_ = find_peaks(sample1.values-mean,distance=100)

In [75]:
fig9 = go.Figure()

fig9.add_trace(go.Scatter(mode = 'lines', line= dict(color='blue'),x=sample1.time, y=sample1.values,name='Raw Data',showlegend=True))
#fig9.add_trace(go.Scatter(mode = 'lines', x=sample1.time, y=mean, name = 'Mean Average', showlegend = True))
fig9.add_trace(go.Scatter(mode = 'lines', x=sample1.time, y=sample1.values-mean, name = 'Mean Subtraction', showlegend = True))
fig9.update_layout(xaxis_range=[79000,85000]);
fig9.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600);

### Interpolating with the mean subtraction peaks

In [76]:
spline=4;
res = 5000
max_x_w, max_y_w, peaks_x_w, peaks_y_w, interpolated_peak_x_w, interpolated_peak_y_w,max_idxs_m = interpolation_spline(mean_peaks, sample1.time, sample1.values, 4, res)

In [77]:
amount_of_peaks = len(max_x_w)

In [78]:
### Comparing peaks of PT with window N

### Comparing Pan Tompkins and mean average substraction + Wavelet Transform with all windows in Moving Window Integration

In [79]:
#Visually missing peaks

fig10 = go.Figure()
missing_mean_p_dict ={};  missing_pt_p_dict={};
spline=4;
res = 5000


for N in lengths: 
    #print(N)
    missing_pt_p, missing_mean_p = find_missing_peaks(max_x_dict['Window: '+str(N)], max_x_w, True)
    missing_pt_p_dict['window_length_'+str(N)] = missing_pt_p
    missing_mean_p_dict['window_length_'+str(N)] = missing_mean_p
    
    fig10.add_trace(go.Scatter(line= dict(color='blue'),x=sample1.time, y=sample1.values,name='Raw Data',showlegend=True, visible = False))
    

    fig10.add_trace(go.Scatter(mode='markers',marker=dict(size=12),x=max_x_w, y= max_y_w, name = 'Mean Peaks', 
                               showlegend=True, visible = False))
    fig10.add_trace(go.Scatter(mode='markers',marker=dict(size=10,color='#21B626'), x = max_x_dict['Window: '+str(N)], 
                              y = max_y_dict['Window: '+str(N)], name = 'P.T. Peaks WL: '+str(N),showlegend=True, 
                              visible = False))
    fig10.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='orange'), x = max_x_dict['Window: '+str(N)][missing_mean_p], 
                               y= max_y_dict['Window: '+str(N)][missing_mean_p], name = 'Missing Mean Peaks', showlegend=True, visible = False))
    
    fig10.add_trace(go.Scatter(mode='markers',marker=dict(size=7,color='#00FCF5'),x = max_x_w[missing_pt_p] , y= max_y_w[missing_pt_p], 
                              name = 'Missing P.T. Peaks', showlegend=True, visible = False))

fig10.data[5].visible = True

steps = []
start = 0
for i in range(0,len(lengths)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig10.data)},
              {"title": "Peaks identified with PT window length "+str(lengths[i])+ '   Amount of Mean + Wavelet Missing Peaks: '+
               str(len(missing_mean_p_dict['window_length_'+str(lengths[i])])) +'   Amount of PT Missing Peaks: '+ 
               str(len(missing_pt_p_dict['window_length_'+str(lengths[i])]))}],  # layout attribute
    )
    step["args"][0]["visible"][start] = True  # Toggle i'th trace to "visible"
    step["args"][0]["visible"][start+1] = True
    step["args"][0]["visible"][start+2] = True
    step["args"][0]["visible"][start+3] = True
    step["args"][0]["visible"][start+4] = True
    
    start = start+5
    steps.append(step)
    
sliders = [dict(active=5, currentvalue={"prefix": "n: "}, pad={"t": 50}, steps=steps)]
fig10.update_layout(sliders=sliders);
fig10.update_layout(xaxis_range=[79000,85000]);
fig10.update_layout(xaxis = dict(tickmode = 'array', tickvals = sample1.ticks_values, ticktext = sample1.time_format_axis),autosize=False,width=1200,height=600); 

In [80]:
#fig10.show();

In [173]:
def overlapping_mean(total_time,heart_beat_time, time_window_length, windows): 
    # This block calculates the mean heartrate of the different windows 
    print('total_time: '+str(total_time) + ' mins')
    print('')
    print('windows: '+ str(windows))

    first_array = np.delete(heart_beat_time,0)
    second_array = np.delete(heart_beat_time,-1)

    dif = (first_array - second_array)/1000
    heart_rate = 60/dif

    # k is the index where the cumulative sum of time is greater than half the time interval
    k = 0
    cmltive_time = 0

    #The even_idx and odd_idx arrays store the indices where the windows start and finish.
    even_idx = []
    odd_idx = [0]
    odd_window_mean = []
    even_window_mean = []

    # The total time of the interval will be defined by the amount of windows one chooses, the overlap and the length of the windows. 
    #This first loop is to find the index of the beginning of the second window 
    while cmltive_time < (time_window_length*60)/2 : 
        k = k + 1
        cmltive_time = np.sum(dif[0:k])
    even_idx = np.append(even_idx,k-1)

    leading_time = time_window_length*60

    window_counter = 1
    odd = True 


    #At the end of the while there is still one more window to calculate its mean.
    while window_counter < windows : 
        while cmltive_time < leading_time: 
            k = k + 1 
            cmltive_time = np.sum(dif[0:k])

        # The reason we append the index k-1 is because when the condition cmltive_time < leading_time isn't met you actually want the index from the sample before where the condition was actually met, 
        # which is k-1. 

        # The 'dif' array is actually the heartrate array. So what we need to do is to consider all of the heartbeats in that array for the established windows. 

        if odd:
            odd = False 
            odd_idx = np.append(odd_idx,k-1)
            odd_idx = np.asarray(odd_idx, dtype = 'int')
            odd_window_mean = np.append(odd_window_mean ,np.mean(heart_rate[odd_idx[-2]:odd_idx[-1]]))

        else: 
            odd = True
            even_idx = np.append(even_idx,k-1)
            even_idx = np.asarray(even_idx, dtype = 'int')
            even_window_mean = np.append(even_window_mean, np.mean(heart_rate[even_idx[-2]:even_idx[-1]]))

        leading_time = (time_window_length + (1-overlap_window)*(time_window_length)*(window_counter))*60
        window_counter = window_counter + 1

    #In this last part we include the mean of the last window. 

    if odd:
        odd_idx=np.append(odd_idx,len(dif))
        odd_window_mean = np.append(odd_window_mean ,np.mean(heart_rate[odd_idx[-2]:odd_idx[-1]]))

    else: 
        even_idx=np.append(even_idx,len(dif))
        even_window_mean = np.append(even_window_mean ,np.mean(heart_rate[even_idx[-2]:even_idx[-1]]))

    overlap_mean_values=odd_window_mean
    adding = 1

    #Joinning the mean odd and even window arrays to a single one in order. 
    for j in range(0, len(even_window_mean)):
        overlap_mean_values = np.insert(overlap_mean_values,j+adding,even_window_mean[j])
        adding = adding + 1
    
    return overlap_mean_values


In [189]:
overlapping_array = overlapping_mean(total_time,max_x_dict['Window: 25'], time_wind_length, windows)
print('')
print('overlapping_array: '+str(overlapping_array))

total_time: 20 mins

windows: 7

overlapping_array: [177.94915155 182.36143809 183.38517608 174.21389    171.08633536
 171.73596706 170.82293089]


## Calculating the mean of overlapping windows 

In [184]:
# This block calculates the mean heartrate of the different windows 
print('total_time: '+str(total_time) + ' mins')
print('')
print('windows: '+ str(windows))

first_array = np.delete(max_x_dict['Window: 25'],0)
second_array = np.delete(max_x_dict['Window: 25'],-1)

dif = (first_array - second_array)/1000
heart_rate = 60/dif

# k is the index where the cumulative sum of time is greater than half the time interval
k = 0
cmltive_time = 0

#The even_idx and odd_idx arrays store the indices where the windows start and finish.
even_idx = []
odd_idx = [0]
odd_window_mean = []
even_window_mean = []

# The total time of the interval will be defined by the amount of windows one chooses, the overlap and the length of the windows. 
#This first loop is to find the index of the beginning of the second window 
while cmltive_time < (time_wind_length*60)/2 : 
    k = k + 1
    cmltive_time = np.sum(dif[0:k])
even_idx = np.append(even_idx,k-1)

leading_time = time_wind_length*60

window_counter = 1
odd = True 


#At the end of the while there is still one more window to calculate its mean.
while window_counter < windows : 
    while cmltive_time < leading_time: 
        k = k + 1 
        cmltive_time = np.sum(dif[0:k])

    # The reason we append the index k-1 is because when the condition cmltive_time < leading_time isn't met you actually want the index from the sample before where the condition was actually met, 
    # which is k-1. 
    
    # The 'dif' array is actually the heartrate array. So what we need to do is to consider all of the heartbeats in that array for the established windows. 
    
    if odd:
        odd = False 
        odd_idx = np.append(odd_idx,k-1)
        odd_idx = np.asarray(odd_idx, dtype = 'int')
        odd_window_mean = np.append(odd_window_mean ,np.mean(heart_rate[odd_idx[-2]:odd_idx[-1]]))

    else: 
        odd = True
        even_idx = np.append(even_idx,k-1)
        even_idx = np.asarray(even_idx, dtype = 'int')
        even_window_mean = np.append(even_window_mean, np.mean(heart_rate[ even_idx[-2] : even_idx[-1] ]))

        

    leading_time = (time_wind_length + (1-overlap_window)*(time_wind_length)*(window_counter))*60
    window_counter = window_counter + 1
    
#In this last part we include the mean of the last window. 

if odd:
    odd_idx=np.append(odd_idx,len(dif))
    odd_window_mean = np.append(odd_window_mean ,np.mean(heart_rate[odd_idx[-2]:odd_idx[-1]]))

else: 
    even_idx=np.append(even_idx,len(dif))
    even_window_mean = np.append(even_window_mean ,np.mean(heart_rate[ even_idx[-2] : even_idx[-1] ]))

overlap_m_values=odd_window_mean
adding = 1

#Joinning the mean odd and even window arrays to a single one in order. 
for j in range(0, len(even_window_mean)):
    overlap_m_values = np.insert(overlap_m_values,j+adding,even_window_mean[j])
    adding = adding + 1
    

total_time: 20 mins

windows: 7
